In [12]:
# Change to project directory
import os
import sys


# Clone the repository using HTTPS URL if it doesn't exist
! git clone https://github.com/KiRorY/VesselSeg_Pytorch_MLSP.git


fatal: destination path 'VesselSeg_Pytorch_MLSP' already exists and is not an empty directory.


In [ ]:

import os
import sys
import time
import random
import numpy as np
import torch
from torch.utils.data import DataLoader
import argparse

# Import custom modules
sys.path.append('/content/VesselSeg_Pytorch_MLSP')
from trainer import (load_raw_drive_data, generate_patches, VesselPatchDataset,
                     train_model, evaluate_model)
from models import get_model
from metrics import MetricsCalculator
from visualization import (plot_training_curves, plot_roc_curves, plot_metrics_comparison,
                           create_metrics_table, plot_radar_chart, plot_segmentation_results,
                           plot_error_maps)
from comparison import (pairwise_comparison, create_summary_table, rank_models,
                       save_results, generate_text_report)


In [14]:



def set_seed(seed):
    """Set random seeds for reproducibility"""
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False



In [15]:

# ============================================================================
# Data Preparation
# ============================================================================

def prepare_data(config):
    """
    Load and prepare dataset

    Returns:
        train_loader, val_loader
    """
    print("\n" + "="*80)
    print("STEP 1: DATA PREPARATION")
    print("="*80)

    # Load raw data
    print("\n1.1 Loading raw DRIVE data...")
    images, masks, fovs = load_raw_drive_data(config['raw_drive_path'])

    if images is None:
        print("ERROR: Failed to load data. Exiting.")
        sys.exit(1)

    print(f"Loaded {len(images)} images successfully.")

    # Generate patches
    print("\n1.2 Generating patches...")
    patch_data = generate_patches(
        images, masks, fovs,
        (config['patch_height'], config['patch_width']),
        config['n_patches'],
        seed=config['seed']
    )

    print(f"Generated {len(patch_data)} patches.")

    # Split into train and validation
    print(f"\n1.3 Splitting data ({config['val_ratio']*100}% for validation)...")
    from sklearn.model_selection import train_test_split

    train_patches, val_patches = train_test_split(
        patch_data,
        test_size=config['val_ratio'],
        random_state=config['seed']
    )

    # Create datasets
    train_dataset = VesselPatchDataset(train_patches, augment=True)
    val_dataset = VesselPatchDataset(val_patches, augment=False)

    # Create dataloaders
    num_workers = max(1, os.cpu_count() // 2)

    train_loader = DataLoader(
        train_dataset,
        batch_size=config['batch_size'],
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=config['batch_size'] * 2,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )

    print(f"\nTrain set: {len(train_dataset)} patches, {len(train_loader)} batches")
    print(f"Val set:   {len(val_dataset)} patches, {len(val_loader)} batches")

    return train_loader, val_loader, val_dataset



In [16]:

# ============================================================================
# Train All Models
# ============================================================================

def train_all_models(config, train_loader, val_loader):
    """
    Train all models specified in config

    Returns:
        trained_models: dict of {model_name: model}
        histories: dict of {model_name: training_history}
    """
    print("\n" + "="*80)
    print("STEP 2: TRAINING ALL MODELS")
    print("="*80)

    device = config['device']
    models_to_train = config['models_to_train']

    trained_models = {}
    histories = {}
    best_metrics_dict = {}

    for model_name in models_to_train:
        print(f"\n{'='*80}")
        print(f"Training: {model_name.upper()}")
        print(f"{'='*80}")

        # Adjust config for EAM models
        model_config = config.copy()
        if 'eam' in model_name.lower():
            model_config['deep_supervision'] = True
        else:
            model_config['deep_supervision'] = False

        # Train model
        start_time = time.time()
        model, history, best_metrics = train_model(
            model_name,
            model_config,
            train_loader,
            val_loader,
            device,
            save_dir=config['checkpoint_dir']
        )
        elapsed = time.time() - start_time

        print(f"\nTraining completed in {elapsed/60:.2f} minutes")

        trained_models[model_name] = model
        histories[model_name] = history
        best_metrics_dict[model_name] = best_metrics

    return trained_models, histories, best_metrics_dict



In [17]:

# ============================================================================
# Evaluate All Models
# ============================================================================

def evaluate_all_models(trained_models, val_loader, config):
    """
    Evaluate all trained models comprehensively

    Returns:
        all_metrics: dict of {model_name: average_metrics}
        all_results: dict of {model_name: detailed_results}
        all_calculators: dict of {model_name: MetricsCalculator}
    """
    print("\n" + "="*80)
    print("STEP 3: COMPREHENSIVE EVALUATION")
    print("="*80)

    device = config['device']

    all_metrics = {}
    all_results = {}
    all_calculators = {}

    for model_name, model in trained_models.items():
        print(f"\n{'='*60}")
        print(f"Evaluating: {model_name.upper()}")
        print(f"{'='*60}")

        metrics, calculator = evaluate_model(model, val_loader, device)

        all_metrics[model_name] = metrics
        all_calculators[model_name] = calculator

        # Also store per-sample results for statistical tests
        all_results[model_name] = {
            'dice': calculator.metrics['dice'],
            'iou': calculator.metrics['iou'],
            'sensitivity': calculator.metrics['sensitivity'],
            'specificity': calculator.metrics['specificity'],
            'precision': calculator.metrics['precision'],
            'boundary_f1': calculator.metrics['boundary_f1'],
            'assd': calculator.metrics['assd']
        }

        # Print summary
        print(f"\nResults for {model_name}:")
        print(f"  Dice:        {metrics['dice']:.4f} ± {metrics['dice_std']:.4f}")
        print(f"  IoU:         {metrics['iou']:.4f} ± {metrics['iou_std']:.4f}")
        print(f"  Sensitivity: {metrics['sensitivity']:.4f} ± {metrics['sensitivity_std']:.4f}")
        print(f"  Specificity: {metrics['specificity']:.4f} ± {metrics['specificity_std']:.4f}")
        print(f"  Boundary F1: {metrics['boundary_f1']:.4f} ± {metrics['boundary_f1_std']:.4f}")
        print(f"  ASSD:        {metrics['assd']:.4f} ± {metrics['assd_std']:.4f}")
        print(f"  AUC:         {metrics['auc']:.4f}")

    return all_metrics, all_results, all_calculators


In [18]:

# ============================================================================
# Generate Visualizations
# ============================================================================

def generate_visualizations(histories, all_metrics, all_calculators, trained_models,
                           val_dataset, config):
    """Generate all visualization plots"""
    print("\n" + "="*80)
    print("STEP 4: GENERATING VISUALIZATIONS")
    print("="*80)

    figures_dir = config['figures_dir']
    os.makedirs(figures_dir, exist_ok=True)

    device = config['device']

    # 1. Training curves
    print("\n4.1 Plotting training curves...")
    plot_training_curves(histories, os.path.join(figures_dir, 'training_curves.png'))

    # 2. Metrics comparison
    print("4.2 Plotting metrics comparison...")
    plot_metrics_comparison(all_metrics, os.path.join(figures_dir, 'metrics_comparison.png'))

    # 3. ROC curves
    print("4.3 Plotting ROC curves...")
    roc_data = {}
    for model_name, calculator in all_calculators.items():
        fpr, tpr, auc_score = calculator.get_roc_curve()
        if fpr is not None:
            roc_data[model_name] = (fpr, tpr, auc_score)

    if roc_data:
        plot_roc_curves(roc_data, os.path.join(figures_dir, 'roc_curves.png'))

    # 4. Radar chart
    print("4.4 Plotting radar chart...")
    plot_radar_chart(all_metrics, os.path.join(figures_dir, 'radar_chart.png'))

    # 5. Metrics table visualization
    print("4.5 Creating metrics table...")
    create_metrics_table(all_metrics, os.path.join(figures_dir, 'metrics_table.png'))

    # 6. Segmentation results visualization
    print("4.6 Generating segmentation results...")
    num_samples = min(5, len(val_dataset))

    # Get random samples
    indices = np.random.choice(len(val_dataset), num_samples, replace=False)

    images = []
    ground_truths = []
    predictions = {model_name: [] for model_name in trained_models.keys()}

    for idx in indices:
        img_tensor, mask = val_dataset[idx]
        img_np = img_tensor.squeeze().numpy()
        mask_np = mask.numpy()

        images.append(img_np)
        ground_truths.append(mask_np)

        # Get predictions from each model
        for model_name, model in trained_models.items():
            model.eval()
            with torch.no_grad():
                img_input = img_tensor.unsqueeze(0).to(device)
                output = model(img_input)

                if isinstance(output, tuple):
                    output = output[0]

                pred = torch.argmax(output, dim=1).squeeze().cpu().numpy()
                predictions[model_name].append(pred)

    plot_segmentation_results(
        images, ground_truths, predictions, list(trained_models.keys()),
        num_samples=num_samples,
        save_path=os.path.join(figures_dir, 'segmentation_results.png')
    )

    # 7. Error maps
    print("4.7 Generating error maps...")
    # Use first sample for error map
    img = images[0]
    gt = ground_truths[0]
    preds = {model_name: predictions[model_name][0] for model_name in trained_models.keys()}

    plot_error_maps(
        img, gt, preds, list(trained_models.keys()),
        save_path=os.path.join(figures_dir, 'error_maps.png')
    )

    print(f"\nAll visualizations saved to {figures_dir}/")



In [19]:

# ============================================================================
# Statistical Analysis
# ============================================================================

def perform_statistical_analysis(all_metrics, all_results, config):
    """Perform statistical comparison and generate reports"""
    print("\n" + "="*80)
    print("STEP 5: STATISTICAL ANALYSIS")
    print("="*80)

    results_dir = config['results_dir']
    os.makedirs(results_dir, exist_ok=True)

    # 1. Pairwise comparison
    print("\n5.1 Performing pairwise statistical comparison...")
    comparison_results = pairwise_comparison(all_results)

    # 2. Create summary tables
    print("\n5.2 Creating summary tables...")
    summary_df = create_summary_table(
        all_metrics,
        save_path=os.path.join(results_dir, 'summary_table.csv')
    )

    # 3. Rank models
    print("\n5.3 Ranking models...")
    ranks = rank_models(all_metrics)
    ranks.to_csv(os.path.join(results_dir, 'model_rankings.csv'))

    # 4. Save all results
    print("\n5.4 Saving results...")
    save_results(all_metrics, all_results, comparison_results, results_dir)

    # 5. Generate comprehensive report
    print("\n5.5 Generating comprehensive report...")
    generate_text_report(
        all_metrics, all_results, comparison_results,
        save_path=os.path.join(results_dir, 'comprehensive_report.txt')
    )

    return comparison_results


# ============================================================================
# Main Experiment Function
# ============================================================================

def run_experiment(config):
    """Run complete experiment pipeline"""
    print("\n" + "="*80)
    print("MEDICAL IMAGE SEGMENTATION - MODEL COMPARISON EXPERIMENT")
    print("="*80)
    print(f"\nDevice: {config['device']}")
    print(f"Models to train: {', '.join(config['models_to_train'])}")
    print(f"Loss function: {config['loss']}")
    print(f"Deep supervision: {config['deep_supervision']}")

    # Early stopping info
    if config['early_stop'] == float('inf'):
        print(f"Early stopping: DISABLED (will train for all {config['epochs']} epochs)")
    else:
        print(f"Early stopping: ENABLED (patience={config['early_stop']} epochs)")

    # Set seed for reproducibility
    set_seed(config['seed'])

    # Create output directories
    for dir_name in ['checkpoint_dir', 'results_dir', 'figures_dir']:
        os.makedirs(config[dir_name], exist_ok=True)

    # Step 1: Prepare data
    train_loader, val_loader, val_dataset = prepare_data(config)

    # Step 2: Train all models
    trained_models, histories, best_metrics_dict = train_all_models(
        config, train_loader, val_loader
    )

    # Step 3: Evaluate all models
    all_metrics, all_results, all_calculators = evaluate_all_models(
        trained_models, val_loader, config
    )

    # Step 4: Generate visualizations
    generate_visualizations(
        histories, all_metrics, all_calculators,
        trained_models, val_dataset, config
    )

    # Step 5: Statistical analysis
    comparison_results = perform_statistical_analysis(
        all_metrics, all_results, config
    )

    # Final summary
    print("\n" + "="*80)
    print("EXPERIMENT COMPLETED SUCCESSFULLY!")
    print("="*80)
    print("\nSummary:")
    print(f"  Checkpoints saved to: {config['checkpoint_dir']}/")
    print(f"  Results saved to:     {config['results_dir']}/")
    print(f"  Figures saved to:     {config['figures_dir']}/")
    print("\nBest performing model:")

    best_model = max(all_metrics.items(), key=lambda x: x[1]['dice'])
    print(f"  {best_model[0].upper()}")
    print(f"  Dice Score: {best_model[1]['dice']:.4f}")
    print(f"  IoU:        {best_model[1]['iou']:.4f}")
    print(f"  AUC:        {best_model[1]['auc']:.4f}")

    print("\n" + "="*80)

    return trained_models, all_metrics, all_results



In [ ]:
def get_config():
    """Get experiment configuration"""
    config = {
        # Data paths
        "raw_drive_path": "/content/VesselSeg_Pytorch_MLSP/datasets/DRIVE",

        # Patch generation
        "patch_height": 64,
        "patch_width": 64,
        "n_patches": 150000,
        "val_ratio": 0.1,

        # Model settings
        "in_channels": 1,
        "classes": 2,

        # Training hyperparameters
        "epochs": 20,
        "batch_size": 64,
        "lr": 0.0005,
        "early_stop": 3,

        # Loss function
        "loss": "focal_dice",  # Options: 'ce', 'dice', 'bce_dice', 'focal_dice'

        # Deep supervision (for EAM models)
        "deep_supervision": True,

        # Output directories
        "checkpoint_dir": "checkpoints",
        "results_dir": "results",
        "figures_dir": "figures",

        # Random seed
        "seed": 2025,

        # Device
        "device": "cuda" if torch.cuda.is_available() else "cpu",

        # Models to train
        "models_to_train": ['unet', 'unet_eam', 'transunet', 'transunet_eam']
    }

    return config

config = get_config()
run_experiment(config)


MEDICAL IMAGE SEGMENTATION - MODEL COMPARISON EXPERIMENT

Device: cuda
Models to train: unet, unet_eam, transunet, transunet_eam
Loss function: focal_dice
Deep supervision: True
Early stopping: ENABLED (patience=3 epochs)

STEP 1: DATA PREPARATION

1.1 Loading raw DRIVE data...
Found 20 training images. Starting preprocessing...


Processing images: 20it [00:00, 55.76it/s]


All images processed and loaded.
Loaded 20 images successfully.

1.2 Generating patches...
Found 4402936 valid patch centers.


Generating patches: 100%|██████████| 150000/150000 [00:00<00:00, 220685.86it/s]


Generated 150000 patches.

1.3 Splitting data (10.0% for validation)...

Train set: 135000 patches, 2110 batches
Val set:   15000 patches, 118 batches

STEP 2: TRAINING ALL MODELS

Training: UNET

Training UNET
Model parameters: 31,036,546

Starting training for 20 epochs...
Early stopping patience: 3 epochs



...

Evaluating: 100%|██████████| 118/118 [00:29<00:00,  4.00it/s]



Results for transunet:
  Dice:        0.9135 ± 0.0783
  IoU:         0.8476 ± 0.1093
  Sensitivity: 0.9083 ± 0.0927
  Specificity: 0.9914 ± 0.0049
  Boundary F1: 0.9769 ± 0.0837
  ASSD:        0.9493 ± 7.5958
  AUC:         0.9976

Evaluating: TRANSUNET_EAM
Running comprehensive evaluation...


Evaluating: 100%|██████████| 118/118 [00:32<00:00,  3.67it/s]



Results for transunet_eam:
  Dice:        0.9080 ± 0.0782
  IoU:         0.8384 ± 0.1091
  Sensitivity: 0.9022 ± 0.0935
  Specificity: 0.9907 ± 0.0053
  Boundary F1: 0.9768 ± 0.0838
  ASSD:        0.9732 ± 7.5978
  AUC:         0.9974

STEP 4: GENERATING VISUALIZATIONS

4.1 Plotting training curves...
Training curves saved to figures/training_curves.png
4.2 Plotting metrics comparison...
Metrics comparison saved to figures/metrics_comparison.png
4.3 Plotting ROC curves...
ROC curves saved to figures/roc_curves.png
4.4 Plotting radar chart...
Radar chart saved to figures/radar_chart.png
4.5 Creating metrics table...
Metrics table saved to figures/metrics_table.png
4.6 Generating segmentation results...
Segmentation results saved to figures/segmentation_results.png
4.7 Generating error maps...
Error maps saved to figures/error_maps.png

All visualizations saved to figures/

STEP 5: STATISTICAL ANALYSIS

5.1 Performing pairwise statistical comparison...

PAIRWISE STATISTICAL COMPARISON



({'unet': UNet(
    (enc1): ConvBlock(
      (conv_block): Sequential(
        (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
      )
    )
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (enc2): ConvBlock(
      (conv_block): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(128, 